In [ ]:
import numpy as np
from scipy.linalg import block_diag
import matplotlib.pyplot as plt

# Number of sensors
M = 12

# Number of nodes
K = 6

# Locations of the nodes with respect to initial node
locations = (np.array([0,0]),np.array([0.45,0.99]),np.array([3.02,0.45]),np.array([5.61,0.90])
             ,np.array([8.03,1.46]),np.array([8.70,0.50]))

# Direction of arrivals
angles = np.array([-14,30,5])*2*(np.pi)/360
K_l = np.array([np.sin(angles), np.cos(angles)])

# Steering vector is implemented. It should have the shape: (12,3)
for j in range(0,len(locations)):
    for i in range(0,len(angles)):
        first_part = np.exp(1j*np.pi*locations[j].dot(K_l[:,i]))
        second_part = np.array([1, np.exp(1j*np.pi*1*np.sin(angles[i]))]) 
        A_k = np.transpose(np.array(first_part*second_part, ndmin=2))
        if i == 0:
            B_k = A_k
        else:
            B_k = np.concatenate((B_k, A_k), axis=1)
        
        if i == len(angles)-1:
            if j == 0:
                A = np.transpose(B_k)
            else:
                A = np.concatenate((A, np.transpose(B_k)), axis=1)

A = np.transpose(A)

MSE = np.zeros(80)

for snr_dB in range(-20,60):
    
    for i in range(500):

        # Signal(A*s) to noise(n) ratio
        received_snr = 10**(snr_dB/20)
        ratio_As_to_s = 1/2.19
        snr = received_snr*ratio_As_to_s
        N_samples = 100

        # Source signal implementation (shape: (3,500))
        # signal = np.random.normal(0,np.sqrt(snr),(3,N_samples)) + 1j*np.random.normal(0,np.sqrt(snr),(3,N_samples))
        w = np.atleast_2d([np.pi/3, np.pi/4, np.pi/5]).T
        signal = snr*np.exp(1j*w*(np.atleast_2d(np.arange(1,N_samples+1))))
        
        # Received signal power on sensors
        signal_power = np.var(A.dot(signal))

        # Noise signal implementation (shape: (12,500))
        noise = np.random.normal(0,np.sqrt(1),(12,N_samples)) + 1j*np.random.normal(0,np.sqrt(1),(12,N_samples))
        
        if i == 0 and snr_dB == 0:
            print("SIGNAL TO NOISE RATIO")
            print(signal_power/np.sqrt(np.var(noise)))

        # Received signal (shape: (12,500))
        z = A.dot(signal) + noise

        # Sample covariance matrix
        R_sample = z.dot(z.conj().T)

        # Eigenvalue and eigenvectors
        w_sample, v_sample = np.linalg.eig(R_sample)
        if i == 0 and snr_dB == -20:
            print()
            print("EIGENVALUES OF SAMPLE COVARIANCE MATRIX")
            print(w_sample[0])
            print(w_sample[1])
            print(w_sample[2])
            print(w_sample[3])

        # Sensor Selection Matrix (shape: (12,6))
        T = np.array([[1,0,0,0,0,0],
                      [1,0,0,0,0,0],
                      [0,1,0,0,0,0],
                      [0,1,0,0,0,0],
                      [0,0,1,0,0,0],
                      [0,0,1,0,0,0],
                      [0,0,0,1,0,0],
                      [0,0,0,1,0,0],
                      [0,0,0,0,1,0],
                      [0,0,0,0,1,0],
                      [0,0,0,0,0,1],
                      [0,0,0,0,0,1]])

        # Push-Sum Matrix (shape: (6,6))
        P = np.array([[0.3,0.15,0.25,0.14,0.22,0.2],
                      [0.2,0.15,0.25,0.23,0.22,0.2],
                      [0.1,0.15,0.2,0.27,0.06,0.3],
                      [0.1,0.15,0.2,0.12,0.35,0.15],
                      [0.1,0.2,0.05,0.14,0.05,0.10],
                      [0.2,0.2,0.05,0.1,0.1,0.05]]) 

        # Weight Vector  (shape: (6,1))
        w = np.atleast_2d([1,1,1,1,1,1]).T

        # Push-Sum Covariance Matrix Estimation
        R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P,10)).dot(T.T), R_sample)
        R_push_denominator = T.dot(np.linalg.matrix_power(P,10)).dot(w).dot(np.ones((1,6))).dot(T.T)

        # Push Sum Covariance Matrix (shape: (12,12))
        R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
        
        if i == 0 and snr_dB == -20:
            print()
            print("                            R_push")
            print("****************************************************************")
            print("****************************************************************")
            print(R_push)
            print()
            print("                           R_sample")
            print("****************************************************************")
            print("****************************************************************")
            print(R_sample)
            print()
            # Check Convergence Status
            print(R_push == R_sample)

        w_push, v_push = np.linalg.eig(R_push)

        # Upper group selection matrix J_up
        J_up = np.kron(np.eye(6),np.array([1,0]))

        # Lower group selection matrix J_down
        J_down = np.kron(np.eye(6),np.array([0,1]))

        # Push-Sum estimated signal eigenvector matrices
        U_s_push = v_push[:,:3]

        # Upper signal eigenvectors
        U_s_up = J_up.dot(U_s_push)

        # Lower signal eigenvectors
        U_s_down = J_down.dot(U_s_push)

        # Matrix including knowledge about DOAs of the source signals
        psi = np.linalg.inv((U_s_up.conj().T).dot(U_s_up)).dot((U_s_up.conj().T)).dot(U_s_down)

        w2, v2 = np.linalg.eig(psi)
        
        doa_1 = np.arcsin(np.angle(w2[0])/np.pi)/(2*np.pi)*360
        doa_2 = np.arcsin(np.angle(w2[1])/np.pi)/(2*np.pi)*360
        doa_3 = np.arcsin(np.angle(w2[2])/np.pi)/(2*np.pi)*360
        
        if i == 0:
            print()
            print("  DOAs of the source signals in degrees with SNR: " + str(snr_dB) )
            print("****************************************************************")
            print("****************************************************************")
            print("DOA of the first source signal:   " + str(doa_1))
            print("DOA of the second source signal:   " + str(doa_2))
            print("DOA of the third source signal:   " + str(doa_3))

        MSE[snr_dB+20] = MSE[snr_dB+20] + 1/3*1/500*((doa_1-(angles[1]*360/(2*np.pi)))**2+(doa_2-(angles[0]*360/(2*np.pi)))**2+(doa_3-(angles[2]*360/(2*np.pi)))**2)


RMSE = np.sqrt(MSE)
plt.plot(np.arange(-20,60), RMSE, 'r--')




EIGENVALUES OF SAMPLE COVARIANCE MATRIX
(343.7638696394443-4.0476010363009574e-15j)
(106.25364573364327-1.7876146828921624e-14j)
(286.7657882594442-7.609397127310724e-15j)
(123.32958060611345-7.990102413275685e-15j)

                            R_push
****************************************************************
****************************************************************
[[ 2.12628724e+02-2.37085480e-16j -2.00673456e+01-7.24118502e+00j
   1.77665283e+00+1.24633938e+01j  2.13142554e+00+8.23461455e+00j
  -7.14956647e+00-1.82099728e+01j -9.15190066e+00+3.47639320e+00j
  -2.80087006e+00-7.23265660e-01j -2.55171479e+01-1.75580148e+01j
  -8.11328905e+00-1.29626665e+01j -1.27285731e+01+3.09938612e+01j
   5.64968427e+00-3.64286114e+00j -4.19722681e+00-1.06450477e+00j]
 [-2.00673456e+01+7.24118502e+00j  2.08929541e+02-2.90497653e-16j
   1.27199611e+00-2.86103119e+01j  4.37479255e-01-9.96775528e+00j
   2.24613737e+00-1.49091682e+01j  1.59136480e+00-8.90287985e+00j
   6.65608576e+00-6.64


  DOAs of the source signals in degrees with SNR: -19
****************************************************************
****************************************************************
DOA of the first source signal:   27.463274800003173
DOA of the second source signal:   -46.33947692331155
DOA of the third source signal:   -71.17122795412354

  DOAs of the source signals in degrees with SNR: -18
****************************************************************
****************************************************************
DOA of the first source signal:   -3.7372142013446794
DOA of the second source signal:   47.971823895901295
DOA of the third source signal:   -53.98049367583855

  DOAs of the source signals in degrees with SNR: -17
****************************************************************
****************************************************************
DOA of the first source signal:   26.707394141068626
DOA of the second source signal:   -2.0861550390139474
DOA of the third

In [ ]:
print(MSE)

In [ ]:
plt.plot(np.arange(-20,60), RMSE, 'r--')